In [1]:
from hello.hello import HelloApp
from hello.analysis.mixingtime import CompiledMixingTime, CompiledMixingSaveAsInfo
import os
import re

path = "C:\\users\\nathan\\documents\\personal\\test\\mixing_time_raw\\"
mixing_test_re = re.compile(r"mt (\d{1,2})\s*rpm (\d+)\.(\d+)")
from pysrc import logger

logger = logger.BuiltinLogger(__name__)

def dl_all_batchnames(ipv4='192.168.1.15'):
    """
    download all batchnames from the given reactor and
    analyze them.
    """
    app = HelloApp(ipv4)
    app.login()
    batches = app.getBatches()
    ids = batches.ids_to_batches.keys()

    try:
        os.makedirs(path)
    except FileExistsError:
        pass

    for id in ids:
        name = batches.getbatchname(id)
        logger.info("Downloading Batch File: (%d) \"%s\"", id, name)
        report = app.getdatareport_bybatchid(id)

        with open(path + name + ".csv", 'wb') as f:
            f.write(report)

def analyze_batches():
    si = CompiledMixingSaveAsInfo(path.replace("_raw", ""))
    c = CompiledMixingTime(saveas_info=si)
    for file in os.listdir(path):
        c.add_csv(path + file)
    c.analyze_all()

def analyze_position_test():
    si = CompiledMixingSaveAsInfo(path.replace("_raw", "_positions"))
    c = CompiledMixingTime(saveas_info=si)
    for file in os.listdir(path):
        m = mixing_test_re.match(file)
        if not m:
            continue
        if m.groups()[0] == "12":
            c.add_csv(path + file)
    c.analyze_all()

def analyze_mixing_time(files):
    si = CompiledMixingSaveAsInfo(path.replace("_raw", "_mixing"))
    c = CompiledMixingTime(saveas_info=si, logger=logger, tests_per_group=5)
    for file in files:
        m = mixing_test_re.match(file)
        if not m:
            continue
        if m.groups()[1] == "5":
            c.add_csv(path + file)
    c.sort_tests(key=lambda t: int(mixing_test_re.match(t.name).groups()[0]))
    c.analyze_all()

def main_161229():
    #dl_all_batchnames()
    analyze_mixing_time(os.listdir(path))

In [2]:
def start_test(rpm, num):
    app = HelloApp('192.168.1.15')
    app.login()
    app.setag(0, rpm)
    app.endbatch()
    bn = "mt %drpm 5.%d" % (rpm, num)
    app.startbatch(bn)
    input("Press enter to start recipe")
    app.runrecipe("mixing time")
    input("Press enter when PV is stable")
    app.login()
    app.endbatch()
    

In [3]:
# for i in range(1, 6):
#     start_test(19, i)

In [5]:
# app = HelloApp('192.168.1.15')
# app.login()
# for rpm in (12, 15, 19):
#     for i in range(1, 6):
#         bn = "mt %drpm 5.%d" % (rpm, i)
#         print("Downloading:", bn)
#         b = app.getdatareport_bybatchname(bn)
#         with open(path + "\\%s.csv" % bn, 'wb') as f:
#             f.write(b)

In [6]:
def download(rpm, num, app=None):
    app = app or HelloApp('192.168.1.15')
    app.login()
    bn = "mt %drpm 5.%d" % (rpm, num)
    b = app.getdatareport_bybatchname(bn)
    with open(path + "\\%s.csv" % bn, 'wb') as f:
        f.write(b)

In [7]:
def main():
    import subprocess
    subprocess.call("tskill.exe excel")
    from time import sleep
    sleep(1)
    main_161229()

1483050733.800420 DEBUG <__init__>: New Test with name mt 12rpm 5.1
1483050734.336338 DEBUG <__init__>: New Test with name mt 12rpm 5.2
1483050734.763990 DEBUG <__init__>: New Test with name mt 12rpm 5.3
1483050735.185144 DEBUG <__init__>: New Test with name mt 12rpm 5.4
1483050735.596487 DEBUG <__init__>: New Test with name mt 12rpm 5.5
1483050736.175681 DEBUG <__init__>: New Test with name mt 15rpm 5.1
1483050736.728664 DEBUG <__init__>: New Test with name mt 15rpm 5.2
1483050737.261067 DEBUG <__init__>: New Test with name mt 15rpm 5.3
1483050737.797298 DEBUG <__init__>: New Test with name mt 15rpm 5.4
1483050738.248740 DEBUG <__init__>: New Test with name mt 15rpm 5.5
1483050738.761536 DEBUG <__init__>: New Test with name mt 19rpm 5.1
1483050739.186842 DEBUG <__init__>: New Test with name mt 19rpm 5.2
1483050739.693189 DEBUG <__init__>: New Test with name mt 19rpm 5.4
1483050740.243158 DEBUG <__init__>: New Test with name mt 19rpm 5.5
1483050740.662185 DEBUG <__init__>: New Test wit

In [ ]:
def finish_formatting():
    from officelib.xllib import *
    for chart in xl.Selection:
        xlcom.FormatAxesScale(chart.Chart, None, 100)

    for i, co in enumerate(xl.Selection):
        row = i // 5
        col = i % 5
        co.Top = row * co.Height + 20
        co.Left = co.Width * col + 20